In [ ]:
# data : https://www.smard.de/home/downloadcenter/download-marktdaten#!?downloadAttributes=%7B%22selectedCategory%22:3,%22selectedSubCategory%22:8,%22selectedRegion%22:%22DE%22,%22from%22:1658613600000,%22to%22:1659563999999,%22selectedFileType%22:%22CSV%22%7D

In [ ]:
import pandas as pd
import pickle
from pathlib import Path


In [ ]:
df_smard = pd.read_csv('../data/energy_data/elect_price_2022.csv', delimiter=';', decimal=',', parse_dates=[['Datum', 'Uhrzeit']])
df_smard.tail()

In [ ]:
ser_energy_price_germany = df_smard[['Datum_Uhrzeit', 'Deutschland/Luxemburg[€/MWh]']]
print(f"Type: {type(ser_energy_price_germany)}")
ser_energy_price_germany.head()

In [ ]:
type(ser_energy_price_germany['Datum_Uhrzeit'])

In [ ]:
mask = ser_energy_price_germany['Datum_Uhrzeit'].duplicated()

print(ser_energy_price_germany[mask])

In [ ]:
ser_energy_price_germany = ser_energy_price_germany.drop_duplicates(subset=['Datum_Uhrzeit'])

In [ ]:
idx = ser_energy_price_germany['Datum_Uhrzeit']
val = ser_energy_price_germany['Deutschland/Luxemburg[€/MWh]']
ser = val.set_axis(idx, axis=0)

In [ ]:
ser.head()

In [ ]:
upsamled_ser = ser.resample('60s').pad()
#TODO: to datetime?? search for resample

In [ ]:
upsamled_ser.tail()

In [ ]:
# Euro/ MWh to Euro/ kWmin
rescaled_ser = upsamled_ser.mul(1/(1000*60))

In [ ]:
rescaled_ser.tail()

In [ ]:
energy_price_lst = rescaled_ser.to_list()

with open('../data/energy_data/elect_price_2022_kwmin.pkl', 'wb') as outfile:
    pickle.dump(energy_price_lst, outfile)

In [ ]:
print(len(energy_price_lst))

